# Projekt końcowy część 3 - feature engineering

# KOMENTARZ: przetrenowałam modele w czwartej części na bazie zawierającej 65 kolumn i 45 kolumn. Nie ma różnicy w efektach uczenia, jednak przy mniejszej bazie zajmuje ono mniej czasu.

W tym pliku nie ma kolumn: "adr_state" oraz "sub_grade" (co oznacza, że nie ma na nich one-hot-encodingu, więc w sumie jest to różnica ok 20 kolumn).

## Instrukcja

1. Dużą część operacji zaliczających się do feature engineeringu znajdziecie tutaj: https://towardsdatascience.com/feature-engineering-for-machine-learning-3a5e293a5114
2. Poza tym, można pokusić się o własne, "eksperckie" cechy - na przykład
  * dodanie kolumny z propozycją wskaźnika finansowego - na przykład stosunek otrzymanej kwoty pożyczki do wnioskowanej
  * dodanie kolumny z informacją czy pożyczka jest konsolidacyjna bądź nie (1 - tak, 0 - nie, a samą informajcę "wydłubać z jednej z dostępnych kolumn)
  * dodanie kolumny z informacją czy pożyczkobiorca bogato opisał cel pożyczki (może ci co opisuja rozwlekle to lepiej spłacają? albo są pewne słowa klucze których pojawienie się w opisie zmniejsza/zwiększa szansę na spłatę?)
  * i wiele innych, które przyjdą Wam do głowy
  

## Zaimportowanie danych, obejrzenie bazy

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_rel, skew
import sklearn as sk
from datetime import date
from matplotlib import rcParams
from sklearn.preprocessing import power_transform # odpowiednik transformacji logarytmicznej --> ma uczynic rozład naszych danych batdziej gaussowskie
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import scipy.stats as stats
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('drive/MyDrive/projekt_koncowy/Loan_data_after_part_2.csv', index_col = 0)
df.head()

,loan_amnt,term,grade,sub_grade,emp_lenght,home_ownership,annual_inc,verification_status,desc,purpose,...,dti,delinq_2yrs,earliest_cr_line,fico_range_low,inq_last_6mths,open_acc,revol_bal,revol_util_in_percent,pub_rec_bankruptcies,loan_status
1,5000.0,36,2,B2,10.0,2,24000.0,1,1,credit_card,...,27.65,0.0,1985-01-01,735.0,1.0,3.0,13648.0,83.7,0.0,1
2,2500.0,60,3,C4,1.0,2,30000.0,2,1,car,...,1.00,0.0,1999-04-01,740.0,5.0,3.0,1687.0,9.4,0.0,0
3,2400.0,36,3,C5,10.0,2,12252.0,0,0,small_business,...,8.72,0.0,2001-11-01,735.0,2.0,2.0,2956.0,98.5,0.0,1
4,10000.0,36,3,C1,10.0,2,49200.0,2,1,other,...,20.00,0.0,1996-02-01,690.0,1.0,10.0,5598.0,21.0,0.0,1
5,3000.0,60,2,B5,1.0,2,80000.0,2,1,other,...,17.94,0.0,1996-01-01,695.0,0.0,15.0,27783.0,53.9,0.0,1


In [ ]:
df.columns

Index(['loan_amnt', 'term', 'grade', 'sub_grade', 'emp_lenght',
       'home_ownership', 'annual_inc', 'verification_status', 'desc',
       'purpose', 'title(consolidation/not)', 'addr_state', 'dti',
       'delinq_2yrs', 'earliest_cr_line', 'fico_range_low', 'inq_last_6mths',
       'open_acc', 'revol_bal', 'revol_util_in_percent',
       'pub_rec_bankruptcies', 'loan_status'],
      dtype='object')

## Extracting year from erliest_cr_line

In [ ]:
df.earliest_cr_line.value_counts()

1999-10-01    390
1998-11-01    387
2000-10-01    367
1998-12-01    363
2000-11-01    340
             ... 
1967-06-01      1
1968-06-01      1
1967-08-01      1
1963-12-01      1
1969-05-01      1
Name: earliest_cr_line, Length: 529, dtype: int64

In [ ]:
df.earliest_cr_line = pd.to_datetime(df['earliest_cr_line'], format='%Y.%m.%d')

In [ ]:
# Wyciągam rok z earliest_cr_line, żeby algorytm łatwiej analizował dane
df['earliest_cr_line_year'] = df['earliest_cr_line'].dt.year

In [ ]:
# Usuwam kolumnę earliest_cr_line - rok powinien wystarczyć
df.drop(['earliest_cr_line'], axis = 1, inplace = True)

## Dodanie kolumny z nowym wskaźnikiem (installment/monthly_inc)

In [ ]:
# dodaję kolumnę: wskazującą jaki % miesięcznych przychodów stanowi rata
installment = round(df['loan_amnt']/df['term'])
monthly_income = df['annual_inc']/12
df['installment_to_inc'] = 100*(installment/monthly_income)

In [ ]:
df['installment_to_inc'].head()

1    6.950000
2    1.680000
3    6.562194
4    6.780488
5    0.750000
Name: installment_to_inc, dtype: float64

In [ ]:
df.drop(['addr_state', 'sub_grade'], axis=1, inplace = True)

## Binning (fico_range_low, loan_amnt, annual_inc, sub_grade, open_acc, revol_bal)

In [ ]:
# łączę rekordy fico_range_flow w 4 grupy, które wyszły mi z analizy w części 2
df['fico_range_low'] = pd.cut(df.fico_range_low, bins=[0,690,715,745, 850], labels=["Fico_Low", "Fico_LowerMid", "Fico_HigherMID","Fico_High"], include_lowest=True)

In [ ]:
# łączę rekordy loan_amnt w 4 grupy, które wyszły mi z analizy w części 2
df['loan_amnt'] = pd.cut(df.loan_amnt, bins=[0, 5500, 10000, 15500, 45000], labels=["Loan_Low", "Loan_LowerMid", "Loan_HigherMID","Loan_High"], include_lowest=True)

In [ ]:
# łączę rekordy annual_inc w 4 grupy, które wyszły mi z analizy w części 2
df['annual_inc'] = pd.cut(df.annual_inc, bins=[0, 40000, 70000, 100000, 390000], labels=["Inc_Low", "Inc_LowerMid", "Inc_HigherMID","Inc_High"], include_lowest=True)

In [ ]:
# sprawdzam kolumnę open_acc czy wymaga binningu
df.open_acc.value_counts().sort_index()

1.0       34
2.0      651
3.0     1566
4.0     2465
5.0     3317
6.0     4112
7.0     4164
8.0     4095
9.0     3850
10.0    3311
11.0    2880
12.0    2342
13.0    2011
14.0    1555
15.0    1252
16.0     999
17.0     782
18.0     568
19.0     417
20.0     314
21.0     266
22.0     162
23.0     110
24.0      82
25.0      61
26.0      37
27.0      28
28.0      27
29.0      13
30.0      16
31.0      11
Name: open_acc, dtype: int64

In [ ]:
# używam qcut dla znalezienia odpowiednich przedziałów - sprawdzam różnicę pomiędzuy podziałem na 4 i na 10 bins
df['open_acc_quartiles'] = pd.qcut(df['open_acc'], q=4)
df['open_acc_deciles'] = pd.qcut(df['open_acc'], q=10, precision=0)

df.head()

,loan_amnt,term,grade,emp_lenght,home_ownership,annual_inc,verification_status,desc,purpose,title(consolidation/not),...,inq_last_6mths,open_acc,revol_bal,revol_util_in_percent,pub_rec_bankruptcies,loan_status,earliest_cr_line_year,installment_to_inc,open_acc_quartiles,open_acc_deciles
1,Loan_Low,36,2,10.0,2,Inc_Low,1,1,credit_card,0,...,1.0,3.0,13648.0,83.7,0.0,1,1985,6.950000,"(0.999, 6.0]","(0.0, 4.0]"
2,Loan_Low,60,3,1.0,2,Inc_Low,2,1,car,0,...,5.0,3.0,1687.0,9.4,0.0,0,1999,1.680000,"(0.999, 6.0]","(0.0, 4.0]"
3,Loan_Low,36,3,10.0,2,Inc_Low,0,0,small_business,0,...,2.0,2.0,2956.0,98.5,0.0,1,2001,6.562194,"(0.999, 6.0]","(0.0, 4.0]"
4,Loan_LowerMid,36,3,10.0,2,Inc_LowerMid,2,1,other,0,...,1.0,10.0,5598.0,21.0,0.0,1,1996,6.780488,"(9.0, 12.0]","(9.0, 10.0]"
5,Loan_Low,60,2,1.0,2,Inc_HigherMID,2,1,other,0,...,0.0,15.0,27783.0,53.9,0.0,1,1996,0.750000,"(12.0, 31.0]","(13.0, 15.0]"


In [ ]:
df['open_acc_quartiles'].value_counts()

(0.999, 6.0]    12145
(6.0, 9.0]      12109
(12.0, 31.0]     8711
(9.0, 12.0]      8533
Name: open_acc_quartiles, dtype: int64

In [ ]:
df['open_acc_deciles'].value_counts()

(4.0, 6.0]      7429
(0.0, 4.0]      4716
(11.0, 13.0]    4353
(6.0, 7.0]      4164
(7.0, 8.0]      4095
(15.0, 31.0]    3893
(8.0, 9.0]      3850
(9.0, 10.0]     3311
(10.0, 11.0]    2880
(13.0, 15.0]    2807
Name: open_acc_deciles, dtype: int64

In [ ]:
# wybieram podział na 4 biny
pd.qcut(df['open_acc'], q=4)

1        (0.999, 6.0]
2        (0.999, 6.0]
3        (0.999, 6.0]
4         (9.0, 12.0]
5        (12.0, 31.0]
             ...     
42472     (9.0, 12.0]
42473    (0.999, 6.0]
42475    (12.0, 31.0]
42478      (6.0, 9.0]
42479     (9.0, 12.0]
Name: open_acc, Length: 41498, dtype: category
Categories (4, interval[float64, right]): [(0.999, 6.0] < (6.0, 9.0] < (9.0, 12.0] < (12.0, 31.0]]

In [ ]:
# łączę rekordy open_acc_quartiles w 4 grupy, które wyszły z użycia qcut
df.open_acc = pd.cut(df.open_acc, bins=[0, 6,9,12,31], labels=["ACC_Low", "ACC_LowerMid", "ACC_HigherMID","ACC_High"], include_lowest=True)

In [ ]:
# usuwam kolumnę open_acc_deciles oraz open_acc_quartiles
df.drop(['open_acc_deciles', 'open_acc_quartiles'], axis = 1, inplace = True)

In [ ]:
# sprawdzam kolumnę revol_bal czy wymaga binningu
df.revol_bal.value_counts().sort_index()

0.0         990
1.0          13
2.0           5
3.0           7
4.0           3
           ... 
123416.0      1
123611.0      1
123720.0      1
123915.0      1
123980.0      1
Name: revol_bal, Length: 22299, dtype: int64

In [ ]:
# używam qcut dla znalezienia odpowiednich przedziałów - sprawdzam różnicę pomiędzuy podziałem na 4 i na 10 bins
df['revol_bal_quartiles'] = pd.qcut(df['revol_bal'], q=4)
df['revol_bal_deciles'] = pd.qcut(df['revol_bal'], q=10, precision=0)

df.head()

,loan_amnt,term,grade,emp_lenght,home_ownership,annual_inc,verification_status,desc,purpose,title(consolidation/not),...,inq_last_6mths,open_acc,revol_bal,revol_util_in_percent,pub_rec_bankruptcies,loan_status,earliest_cr_line_year,installment_to_inc,revol_bal_quartiles,revol_bal_deciles
1,Loan_Low,36,2,10.0,2,Inc_Low,1,1,credit_card,0,...,1.0,ACC_Low,13648.0,83.7,0.0,1,1985,6.950000,"(8835.0, 17089.5]","(11519.0, 14903.0]"
2,Loan_Low,60,3,1.0,2,Inc_Low,2,1,car,0,...,5.0,ACC_Low,1687.0,9.4,0.0,0,1999,1.680000,"(-0.001, 3689.25]","(1112.0, 2816.0]"
3,Loan_Low,36,3,10.0,2,Inc_Low,0,0,small_business,0,...,2.0,ACC_Low,2956.0,98.5,0.0,1,2001,6.562194,"(-0.001, 3689.25]","(2816.0, 4609.0]"
4,Loan_LowerMid,36,3,10.0,2,Inc_LowerMid,2,1,other,0,...,1.0,ACC_HigherMID,5598.0,21.0,0.0,1,1996,6.780488,"(3689.25, 8835.0]","(4609.0, 6576.0]"
5,Loan_Low,60,2,1.0,2,Inc_HigherMID,2,1,other,0,...,0.0,ACC_High,27783.0,53.9,0.0,1,1996,0.750000,"(17089.5, 123980.0]","(19867.0, 29515.0]"


In [ ]:
df['revol_bal_deciles'].value_counts()

(-1.0, 1112.0]         4152
(8835.0, 11519.0]      4151
(2816.0, 4609.0]       4150
(6576.0, 8835.0]       4150
(14903.0, 19867.0]     4150
(19867.0, 29515.0]     4150
(29515.0, 123980.0]    4150
(4609.0, 6576.0]       4149
(1112.0, 2816.0]       4148
(11519.0, 14903.0]     4148
Name: revol_bal_deciles, dtype: int64

In [ ]:
df['revol_bal_quartiles'].value_counts()

(-0.001, 3689.25]      10375
(17089.5, 123980.0]    10375
(3689.25, 8835.0]      10374
(8835.0, 17089.5]      10374
Name: revol_bal_quartiles, dtype: int64

In [ ]:
# wybieram podział na 4 biny
pd.qcut(df['revol_bal'], q=4)

1          (8835.0, 17089.5]
2          (-0.001, 3689.25]
3          (-0.001, 3689.25]
4          (3689.25, 8835.0]
5        (17089.5, 123980.0]
                ...         
42472      (8835.0, 17089.5]
42473      (-0.001, 3689.25]
42475      (8835.0, 17089.5]
42478      (-0.001, 3689.25]
42479      (-0.001, 3689.25]
Name: revol_bal, Length: 41498, dtype: category
Categories (4, interval[float64, right]): [(-0.001, 3689.25] < (3689.25, 8835.0] < (8835.0, 17089.5] <
                                           (17089.5, 123980.0]]

In [ ]:
# łączę rekordy revol_bal_quartiles w 4 grupy, które wyszły z użycia qcut
df.revol_bal = pd.cut(df.revol_bal, bins=[0, 3689.26,8835.1,17089.6,123980], labels=["revol_bal_Low", "revol_bal_LowerMid", "revol_bal_HigherMID","revol_bal_High"], include_lowest=True)

In [ ]:
# usuwam kolumnę revol_bal_deciles oraz revol_bal_quartiles
df.drop(['revol_bal_deciles', 'revol_bal_quartiles'], axis = 1, inplace = True)

In [ ]:
# sprawdzam czy zrobić binning na inq_last_6mths
df.inq_last_6mths.value_counts()

0.0    19285
1.0    11024
2.0     5878
3.0     3105
4.0     1012
5.0      579
6.0      328
7.0      174
8.0      113
Name: inq_last_6mths, dtype: int64

In [ ]:
# łączę rekordy inq_last_6mths_flow w 4 grupy
df['inq_last_6mths'] = pd.cut(df.inq_last_6mths, bins=[0,2,3,9], labels=["inquire_Low", "inquire_Mid", "inquire_High"], include_lowest=True)

## One hot encoding - purpose, addr_state i wszystkie kolumny po binningu

In [ ]:
df.purpose.value_counts()

debt_consolidation    19434
credit_card            5351
other                  4266
home_improvement       3089
major_purchase         2250
small_business         1906
car                    1587
wedding                 978
medical                 727
moving                  614
house                   406
educational             396
vacation                393
renewable_energy        101
Name: purpose, dtype: int64

In [ ]:
# zmniejszam liczbę kategorii, tak aby te występujące najrzadziej stanowiły jedną grupę - other_purpose
mask = df['purpose'].str.contains(pat= ('wedding|medical|moving|house|educational|vactaion|renewable_energy|other'), na=True)
df.loc[mask, 'purpose'] = 'other_purpose'

In [ ]:
df.purpose.value_counts()

debt_consolidation    19434
other_purpose          7488
credit_card            5351
home_improvement       3089
major_purchase         2250
small_business         1906
car                    1587
vacation                393
Name: purpose, dtype: int64

Poniżej robię one hot encoding na kolejnych kolumnach

In [ ]:
encoded_columns = pd.get_dummies(df['purpose'])
df = df.join(encoded_columns).drop('purpose', axis=1)

In [ ]:
encoded_columns = pd.get_dummies(df['fico_range_low'])
df = df.join(encoded_columns).drop('fico_range_low', axis=1)

In [ ]:
encoded_columns = pd.get_dummies(df['annual_inc'])
df = df.join(encoded_columns).drop('annual_inc', axis=1)

In [ ]:
encoded_columns = pd.get_dummies(df['loan_amnt'])
df = df.join(encoded_columns).drop('loan_amnt', axis=1)

In [ ]:
encoded_columns = pd.get_dummies(df['open_acc'])
df = df.join(encoded_columns).drop('open_acc', axis=1)

In [ ]:
encoded_columns = pd.get_dummies(df['inq_last_6mths'])
df = df.join(encoded_columns).drop('inq_last_6mths', axis=1)

In [ ]:
encoded_columns = pd.get_dummies(df['revol_bal'])
df = df.join(encoded_columns).drop('revol_bal', axis=1)

## Obejrzenie i zapisanie bazy po zmianach.

In [ ]:
df.columns

Index(['term', 'grade', 'emp_lenght', 'home_ownership', 'verification_status',
       'desc', 'title(consolidation/not)', 'dti', 'delinq_2yrs',
       'revol_util_in_percent', 'pub_rec_bankruptcies', 'loan_status',
       'earliest_cr_line_year', 'installment_to_inc', 'car', 'credit_card',
       'debt_consolidation', 'home_improvement', 'major_purchase',
       'other_purpose', 'small_business', 'vacation', 'Fico_Low',
       'Fico_LowerMid', 'Fico_HigherMID', 'Fico_High', 'Inc_Low',
       'Inc_LowerMid', 'Inc_HigherMID', 'Inc_High', 'Loan_Low',
       'Loan_LowerMid', 'Loan_HigherMID', 'Loan_High', 'ACC_Low',
       'ACC_LowerMid', 'ACC_HigherMID', 'ACC_High', 'inquire_Low',
       'inquire_Mid', 'inquire_High', 'revol_bal_Low', 'revol_bal_LowerMid',
       'revol_bal_HigherMID', 'revol_bal_High'],
      dtype='object')

In [ ]:
df.shape

(41498, 45)

In [ ]:
df.head()

,term,grade,emp_lenght,home_ownership,verification_status,desc,title(consolidation/not),dti,delinq_2yrs,revol_util_in_percent,...,ACC_LowerMid,ACC_HigherMID,ACC_High,inquire_Low,inquire_Mid,inquire_High,revol_bal_Low,revol_bal_LowerMid,revol_bal_HigherMID,revol_bal_High
1,36,2,10.0,2,1,1,0,27.65,0.0,83.7,...,0,0,0,1,0,0,0,0,1,0
2,60,3,1.0,2,2,1,0,1.00,0.0,9.4,...,0,0,0,0,0,1,1,0,0,0
3,36,3,10.0,2,0,0,0,8.72,0.0,98.5,...,0,0,0,1,0,0,1,0,0,0
4,36,3,10.0,2,2,1,0,20.00,0.0,21.0,...,0,1,0,1,0,0,0,1,0,0
5,60,2,1.0,2,2,1,0,17.94,0.0,53.9,...,0,0,1,1,0,0,0,0,0,1


In [ ]:
path = '/content/drive/My Drive/projekt_koncowy/Loan_data_after_part_3_mniej_kolumn.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
  df.to_csv(f)